## Building the LeNet-5 Network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# The Class for LeNet-5

In [2]:
# let us set up the Lenet Class
class Lenet5(nn.Module):
    def __init__(self, num_channels, num_classes):
        super(Lenet5, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=6, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0))
        self.max_pool = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride=(1 ,1), padding=(0, 0))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0))
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)
    
    def forward(self, x):
        x = torch.tanh(self.max_pool(torch.tanh(self.conv1(x))))
        x = torch.tanh(self.max_pool(torch.tanh(self.conv2(x))))
        x = torch.tanh(self.conv3(x))
        x = x.flatten(start_dim=1)
        x = torch.tanh(self.fc1(x))
        x = torch.softmax(self.fc2(x), dim=1)
        return x

### *Sanity check for LeNet-5*

In [3]:
# lets do a sanity test on LeNet object
lenet5_test = Lenet5(1, 10)
random_input_batch = torch.randn(512, 1, 32, 32)
print(lenet5_test(random_input_batch).shape)

torch.Size([512, 10])


## Computing mean and std for our training dataset

In [4]:
# let us set up the training dataset
mean_std_data = datasets.MNIST(root='./data',
                                   download=True,
                                   train=True,
                                   transform=transforms.Compose([
                                       transforms.Pad(padding=2, fill=0, padding_mode='constant'),
                                       transforms.ToTensor(),
                                   ]))

# lets compute the mean and std to normalize our training dataset later on.
# (MNIST is small enough to do mean, std computation this way)
mean_std_loader = DataLoader(dataset=mean_std_data,
                             batch_size=len(mean_std_data))
data = next(iter(mean_std_loader))
train_mean = data[0].mean()
train_std = data[0].std()
train_mean, train_std

(tensor(0.1000), tensor(0.2752))

## Loading in data with transforms used in LeNet

In [5]:
"""
    Since LeNet requires 32 x 32 images as per the paper, 
    the MNIST images were obviously padded (since their dimensions are 28 x 28).
    The reason that LeCun cites in the paper is that its better for the stroke-ends 
    and corners to be centered in the receptive field of the high level feature detectors
    (probably didn't go for a bigger field size because the largest character in the mnist 
    dataset has a size of 20 x 20).
"""

# let us set up the training dataset
train_data = datasets.MNIST(root='./data',
                                   download=True,
                                   train=True,
                                   transform=transforms.Compose([
                                       transforms.Pad(padding=2, fill=0, padding_mode='constant'),
                                       transforms.ToTensor(),
                                       transforms.Normalize((train_mean,), (train_std,))
                                   ]))
# and now the dataloader
train_data_loader = DataLoader(dataset=train_data,
                               shuffle=True,
                               batch_size=512,
                               num_workers=2)

### *Sanity Check for train data loader*

In [6]:
# sanity check: Check dimensions of a single batch of data
for idx, (data, target) in enumerate(train_data_loader):
    print(data.shape)
    break

torch.Size([512, 1, 32, 32])


## Training Loop

In [7]:
# set up device, hyperparameters and the training loop
device = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CHANNELS=1
NUM_CLASSES=10

lenet5 = Lenet5(NUM_CHANNELS, NUM_CLASSES).to(device=device)
loss_criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lenet5.parameters(), lr=0.001)

for epoch in range(20):
    for idx, (data, target) in enumerate(train_data_loader):
        data = data.to(device=device)
        target = target.to(device=device)
        outputs = lenet5(data)

        loss = loss_criterion(outputs, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: {}, loss: {}".format(epoch + 1, loss))
print("Done Learning!")

Epoch: 1, loss: 1.584311604499817
Epoch: 2, loss: 1.4870234727859497
Epoch: 3, loss: 1.511109471321106
Epoch: 4, loss: 1.5001393556594849
Epoch: 5, loss: 1.4633334875106812
Epoch: 6, loss: 1.4766794443130493
Epoch: 7, loss: 1.4706932306289673
Epoch: 8, loss: 1.4775718450546265
Epoch: 9, loss: 1.4733105897903442
Epoch: 10, loss: 1.4676822423934937
Epoch: 11, loss: 1.4747141599655151
Epoch: 12, loss: 1.4788789749145508
Epoch: 13, loss: 1.463736653327942
Epoch: 14, loss: 1.4723554849624634
Epoch: 15, loss: 1.4745911359786987
Epoch: 16, loss: 1.4618431329727173
Epoch: 17, loss: 1.4613240957260132
Epoch: 18, loss: 1.4646424055099487
Epoch: 19, loss: 1.4699339866638184
Epoch: 20, loss: 1.4721182584762573
Done Learning!


## Computing Mean, std for test data (and loading in data)

In [8]:
# let us set up the testing dataset for mean, std computation
mean_std_data_test = datasets.MNIST(root='./data',
                                   download=True,
                                   train=False,
                                   transform=transforms.Compose([
                                       transforms.Pad(padding=2, fill=0, padding_mode='constant'),
                                       transforms.ToTensor(),
                                   ]))

# lets compute the mean and std to normalize our testing dataset later on 
# (MNIST is small enough to do mean, std computation this way)
mean_std_loader_test = DataLoader(dataset=mean_std_data_test,
                             batch_size=len(mean_std_data_test))
data = next(iter(mean_std_loader_test))
test_mean = data[0].mean()
test_std = data[0].std()
test_mean, test_std

(tensor(0.1015), tensor(0.2774))

In [9]:
# set up test data and test data loader
test_data = datasets.MNIST(root='./data', 
                                  train=False,
                                  download=True,
                                  transform=transforms.Compose([
                                      transforms.Pad(padding=2, fill=0, padding_mode='constant'),
                                      transforms.ToTensor(),
                                      transforms.Normalize((test_mean,), (test_std,))
                                  ]))

test_data_loader = DataLoader(dataset=test_data,
                              shuffle=True,
                              batch_size=512)

## Testing Loop

In [10]:
#98.28 with tanh after maxpool (no normalization)
#98.08 without tanh after maxpool (no normalization)
#98.88 with tanh after maxpool (normalized)
#98.59 without tanh after maxpool(normalized)
num_correct = 0
num_samples = 0

lenet5.eval()
# disable gradient computation for test set
with torch.no_grad():
    for idx, (data, targets) in enumerate(test_data_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        outputs = lenet5(data)
        values, idx_of_max_value = outputs.max(1)
        num_correct += (idx_of_max_value == targets).sum()
        num_samples += targets.shape[0]
    print(num_correct, num_samples)
    print("Accuracy of model on the test set: {}".format((num_correct.item() / num_samples) * 100))

tensor(9888, device='cuda:0') 10000
Accuracy of model on the test set: 98.88
